# GMC - Solid Electrolyte Interphase 
Simple example of [solid-electrolyte interphase formation and evolution in a lithium-ion battery](https://pubs.acs.org/doi/10.1021/acsenergylett.2c00517).

Note: This example requires OpenBabel (```conda install -c conda-forge openbabel```)

In [ ]:
import sqlite3
from itertools import combinations
import os

import numpy as np
from scipy.constants import pi, epsilon_0, elementary_charge

from monty.serialization import loadfn

from pymatgen.analysis.graphs import MoleculeGraph
from pymatgen.analysis.local_env import OpenBabelNN, metal_edge_extender

In [ ]:
############ USER DEFINED #################
# location of output initial_state.sqlite #
# and rn.sqlite files                     #
base_dir = ""                             #
###########################################

# Room temperature (25 C) in Kelvin
ROOM_TEMP = 298.15
# ROOM_TEMP = 423.15

# Boltzmann constant in eV / K
KB = 8.617333262 * 10 ** -5

# Planck constant in eV * s
PLANCK = 4.135667696 * 10 ** -15

# Simulation variables
electron_free_energy = -1.4
dielectric = 18.5
refractive = 1.415
radius = 5.0
decay_constant=1.2

# Residence time of Li+ with EC, from Tingzheng, is roughly 5 ns (0.266 eV barrier)
min_hopping_barrier = 0.266
# min_hopping_barrier = 0.325

# kMC parameters
factor_zero = 1.0
factor_two = 1.0
factor_duplicate = 0.5

In [ ]:
def eyring(dg_barrier, kappa=1.0, temperature=ROOM_TEMP):
    if dg_barrier <= 0:
        return kappa * KB * temperature / PLANCK
    else:
        return kappa * KB * temperature / PLANCK * np.exp(-dg_barrier / (KB * temperature))


def lambda_inner(a_a, b_b, a_b, b_a):
    return ((a_b - a_a) + (b_a - b_b)) / 2 * 27.2114


def lambda_outer(r=radius, d=radius, eps=dielectric, n=refractive):
    lambda_o = abs(elementary_charge) / (8 * pi * epsilon_0)
    lambda_o *= (1 / r - 1 / (2 * d)) * 10 ** 10
    lambda_o *= 1 / n ** 2 - 1 / eps
    return lambda_o


def barrier_redox(dg, a_a, b_b, a_b, b_a, dq=1, e_free=electron_free_energy, r=radius, d=radius, eps=dielectric, n=refractive):
    lambda_i = lambda_inner(a_a, b_b, a_b, b_a)
    lambda_o = lambda_outer(r=r, d=d, eps=eps, n=n)
    lambda_total = lambda_i + lambda_o

    delta_g = dg + dq * e_free

    dg_barrier = lambda_total / 4 * (1 + delta_g / lambda_total) ** 2

    return dg_barrier

def get_thermo(reaction, mols, mapping, ts):
    t = ts[reaction["ts"]]["free_energy"]
    reactants = sum([mols[mapping[r]]["free_energy"] for r in reaction["reactants"]])
    products = sum([mols[mapping[r]]["free_energy"] for r in reaction["products"]])

    return {"dg": products - reactants,
            "forward_barrier": t - reactants,
            "reverse_barrier": t - products}

In [ ]:
create_metadata_table = """
    CREATE TABLE metadata (
            number_of_species   INTEGER NOT NULL,
            number_of_reactions INTEGER NOT NULL
    );
"""

insert_metadata = """
    INSERT INTO metadata VALUES (?, ?)
"""

# it is important that reaction_id is the primary key
# otherwise the network loader will be extremely slow.
create_reactions_table = """
    CREATE TABLE reactions (
            reaction_id         INTEGER NOT NULL PRIMARY KEY,
            number_of_reactants INTEGER NOT NULL,
            number_of_products  INTEGER NOT NULL,
            reactant_1          INTEGER NOT NULL,
            reactant_2          INTEGER NOT NULL,
            product_1           INTEGER NOT NULL,
            product_2           INTEGER NOT NULL,
            rate                REAL NOT NULL,
            dG                  REAL NOT NULL,
            dG_barrier          REAL NOT NULL,
            is_redox            INTEGER NOT NULL
    );
"""

insert_reaction = """
    INSERT INTO reactions VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
"""

create_initial_state_table = """
    CREATE TABLE initial_state (
            species_id             INTEGER NOT NULL PRIMARY KEY,
            count                  INTEGER NOT NULL
    );
"""

create_trajectories_table = """
    CREATE TABLE trajectories (
            seed         INTEGER NOT NULL,
            step         INTEGER NOT NULL,
            reaction_id  INTEGER NOT NULL,
            time         REAL NOT NULL
    );
"""

create_factors_table = """
    CREATE TABLE factors (
            factor_zero      REAL NOT NULL,
            factor_two       REAL NOT NULL,
            factor_duplicate REAL NOT NULL
    );
"""

create_interrupt_state_table = """
    CREATE TABLE interrupt_state (
      seed                    INTEGER NOT NULL,
      species_id              INTEGER NOT NULL,
      count                   INTEGER NOT NULL
);
"""

create_interrupt_cutoff_table = """
    CREATE TABLE interrupt_cutoff (
      seed                    INTEGER NOT NULL,
      step                    INTEGER NOT NULL,
      time                    INTEGER NOT NULL       
);
"""

In [ ]:
marcus_sp = loadfn("20211130_marcus_sp.json")
marcus_mols = [(e["orig"]["molecule"]) for e in marcus_sp]
marcus_mgs = [metal_edge_extender(MoleculeGraph.with_local_env_strategy(m, OpenBabelNN())) for m in marcus_mols]

mols = loadfn("20211130_mol_entries.json")
mapping = {m["name"]: i for i, m in enumerate(mols)}

ec_ind = mapping["EC"]
li_plus_ind = mapping["Li_plus"]
oh_minus_ind = mapping["OH_minus"]
h_ind = mapping["H"]
co2_ind = mapping["CO2"]
liec_ind = mapping["LiEC_plus"]
ledc_ind = mapping["LEDC"]
lemc_ind = mapping["LEMC"]
carbonate_ind = mapping["Li2CO3"]
oxalate_ind = mapping["Li_oxalate"]

ts_normal = loadfn("20211130_ts.json")
ts_lowlot = loadfn("20211130_ts_lowlot.json")

forbidden_reactions = [(30, 24, 8, -1),
                       (23, -1, 24, 31)]

In [ ]:
# for electron_free_energy in [-1.4, -1.5, -1.6, -1.7, -1.8, -1.9, -2.0, -2.1]:
for electron_free_energy in [-1.4]:
    # for electrode_distance in [0.0, 10.0, 20.0]:
    for electrode_distance in [0.0]:
        if electrode_distance == 0.0:
            adiabatic = True
        else:
            adiabatic = False

        # for rate_reduction in [10, 100, 1000]:
        reaction_index = 0
        reactions = set()

        name = "e{}V_dist{}A_co250_h2o10".format(round(-1 * (electron_free_energy + 1.4), 1), electrode_distance)
        os.mkdir(os.path.join(base_dir, name))

        rn_con = sqlite3.connect(os.path.join(base_dir, name, "rn.sqlite"))
        rn_cur = rn_con.cursor()
        rn_cur.execute(create_metadata_table)
        rn_cur.execute(create_reactions_table)
        rn_con.commit()

        print("ADDING REDOX REACTIONS")
        # Identify reduction/oxidation reactions
        for ii, mol1 in enumerate(mols):
            if ii == 0:
                continue

            for jj, mol2 in enumerate(mols[:ii]):
                if mol1["formula"] != mol2["formula"]:
                    continue
                if mol1["charge"] == mol2["charge"]:
                    continue

                dq = mol2["charge"] - mol1["charge"]

                if abs(dq) != 1:
                    continue

                if not mol1["mg_noli"].isomorphic_to(mol2["mg_noli"]):
                    continue

                # Match found - so there should be a redox reaction
                dg = mol2["free_energy"] - mol1["free_energy"]

                a_a = mol1["energy"]
                b_b = mol2["energy"]

                a_b = None
                b_a = None

                for kk, mm in enumerate(marcus_mgs):
                    if mol1["molecule_graph"].isomorphic_to(mm) and mm.molecule.charge == mol2["charge"]:
                        a_b = marcus_sp[kk]["output"]["final_energy"]
                    elif mol2["molecule_graph"].isomorphic_to(mm) and mm.molecule.charge == mol1["charge"]:
                        b_a = marcus_sp[kk]["output"]["final_energy"]
                    if a_b is not None and b_a is not None:
                        break

                if a_b is None or b_a is None:
                    print("MISSING SP", ii, jj)
                    continue

                if adiabatic:
                    barrier_forward = barrier_redox(dg, a_a, b_b, a_b, b_a, dq=dq, d=radius, e_free=electron_free_energy)
                    barrier_reverse = barrier_redox(-1 * dg, b_b, a_a, b_a, a_b, dq=(-1 * dq), d=radius, e_free=electron_free_energy)

                    k_forward = eyring(barrier_forward)
                    k_reverse = eyring(barrier_reverse)
                else:
                    barrier_forward = barrier_redox(dg, a_a, b_b, a_b, b_a, dq=dq, d=electrode_distance, e_free=electron_free_energy)
                    barrier_reverse = barrier_redox(-1 * dg, b_b, a_a, b_a, a_b, dq=(-1 * dq), d=electrode_distance, e_free=electron_free_energy)

                    kappa = np.exp(-1 * electrode_distance * decay_constant)

                    k_forward = eyring(barrier_forward, kappa=kappa)
                    k_reverse = eyring(barrier_reverse, kappa=kappa)

                print("REDOX:", ii, jj, dg, barrier_forward, barrier_reverse, k_forward, k_reverse)

                if (ii, -1, jj, -1) not in reactions:
                    # Forward reaction
                    reactions.add((ii, -1, jj, -1))
                    rn_cur.execute(insert_reaction,
                                   (reaction_index,
                                    1,
                                    1,
                                    ii,
                                    -1,
                                    jj,
                                    -1,
                                    k_forward,
                                    dg,
                                    barrier_forward,
                                    True)
                                   )
                    reaction_index += 1
                    rn_con.commit()

                if (jj, -1, ii, -1) not in reactions:
                    reactions.add((jj, -1, ii, -1))
                    # Reverse reaction
                    rn_cur.execute(insert_reaction,
                                   (reaction_index ,
                                    1,
                                    1,
                                    jj,
                                    -1,
                                    ii,
                                    -1,
                                    k_reverse,
                                    -1 * dg,
                                    barrier_reverse,
                                    True)
                                   )
                    reaction_index += 1
                    rn_con.commit()
        print("DONE ADDING REDOX; {} TOTAL REACTIONS".format(reaction_index))

        print("ADDING COORDINATION REACTIONS")
        # Next, identify all coordination and metal-hopping reactions
        # First, just identify pairs of molecules that are different only by a change in Li
        # Add coordination reactions for all of these
        coordination_pairs = list()
        li_ind, li_entry = [(i, e) for i, e in enumerate(mols) if e["formula"] == "Li1" and e["charge"] == 1][0]
        for ii, mol1 in enumerate(mols):
            if ii == 0:
                continue

            for jj, mol2 in enumerate(mols[:ii]):
                num_li_1 = len([e for e in mol1["molecule"].species if str(e) == "Li"])
                num_li_2 = len([e for e in mol2["molecule"].species if str(e) == "Li"])

                dli = num_li_2 - num_li_1
                dq = mol2["charge"] - mol1["charge"]

                if abs(dli) != 1:
                    continue

                # Assuming all Li are Li+, each additional Li should lead to +1 charge
                if dq != dli:
                    continue

                if not mol1["mg_noli"].isomorphic_to(mol2["mg_noli"]):
                    continue

                # Will use this later to add metal-hopping reactions
                if dli == 1:
                    coordination_pairs.append((ii, jj))
                    dg = mol2["free_energy"] - mol1["free_energy"] - li_entry["free_energy"]
                    reactants = sorted([ii, li_ind])
                    products = [jj, -1]
                    num_reactants = 2
                    num_products = 1
                else:
                    coordination_pairs.append((jj, ii))
                    dg = mol2["free_energy"] + li_entry["free_energy"] - mol1["free_energy"]
                    reactants = [ii, -1]
                    products = sorted([jj, li_ind])
                    num_reactants = 1
                    num_products = 2

                if dg < 0:
                    k_forward = eyring(0)
                    k_reverse = eyring(-1 * dg)
                else:
                    k_forward = eyring(dg)
                    k_reverse = eyring(-1 * dg)

                if (reactants[0], reactants[1], products[0], products[1]) not in reactions:
                    reactions.add((reactants[0], reactants[1], products[0], products[1]))
                    # Forward reaction
                    rn_cur.execute(insert_reaction,
                                   (reaction_index,
                                    num_reactants,
                                    num_products,
                                    reactants[0],
                                    reactants[1],
                                    products[0],
                                    products[1],
                                    k_forward,
                                    dg,
                                    dg,
                                    False)
                                   )
                    reaction_index += 1
                    rn_con.commit()

                if (products[0], products[1], reactants[0], reactants[1]) not in reactions:
                    # Reverse reaction
                    reactions.add((products[0], products[1], reactants[0], reactants[1]))
                    rn_cur.execute(insert_reaction,
                                   (reaction_index,
                                    num_products,
                                    num_reactants,
                                    products[0],
                                    products[1],
                                    reactants[0],
                                    reactants[1],
                                    k_reverse,
                                    -1 * dg,
                                    -1 * dg,
                                    False)
                                   )
                    reaction_index += 1
                    rn_con.commit()

        print("DONE ADDING COORDINATION; {} TOTAL REACTIONS".format(reaction_index))

        print("ADDING METAL HOPPING REACTIONS")
        for combo in combinations(coordination_pairs, 2):
            reactants = sorted([combo[0][0], combo[1][1]])
            products = sorted([combo[1][0], combo[0][1]])
        
            dg = sum([mols[x]["free_energy"] for x in products]) - sum([mols[x]["free_energy"] for x in reactants])
        
            if dg < min_hopping_barrier:
                forward_barrier = min_hopping_barrier
            else:
                forward_barrier = dg
        
            if (-1 * dg) < min_hopping_barrier:
                reverse_barrier = min_hopping_barrier
            else:
                reverse_barrier = -1 * dg
        
            k_forward = eyring(forward_barrier)
            k_reverse = eyring(reverse_barrier)
        
            if (reactants[0], reactants[1], products[0], products[1]) not in reactions:
                reactions.add((reactants[0], reactants[1], products[0], products[1]))
        
                # Forward reaction
                rn_cur.execute(insert_reaction,
                               (reaction_index,
                                2,
                                2,
                                reactants[0],
                                reactants[1],
                                products[0],
                                products[1],
                                k_forward,
                                dg,
                                forward_barrier,
                                False)
                               )
                reaction_index += 1
                rn_con.commit()
        
            if (products[0], products[1], reactants[0], reactants[1]) not in reactions:
                reactions.add((products[0], products[1], reactants[0], reactants[1]))
        
                # Reverse reaction
                rn_cur.execute(insert_reaction,
                               (reaction_index,
                                2,
                                2,
                                products[0],
                                products[1],
                                reactants[0],
                                reactants[1],
                                k_reverse,
                                -1 * dg,
                                reverse_barrier,
                                False)
                               )
                reaction_index += 1
                rn_con.commit()
        
        print("DONE ADDING METAL HOPPING; {} TOTAL REACTIONS".format(reaction_index))


        ts_reacts = [{"ts": "LiEC_bi___LiEC_RO1",
                      "reactants": ["LiEC"],
                      "products": ["LiEC_RO1"]},
                     {"ts": "LiEC_RO1___C2H4__LiCO3",
                      "reactants": ["LiEC_RO1"],
                      "products": ["C2H4", "LiCO3"]},
                     {"ts": "LiEC_RO1___C2H4__LiCO3_minus",
                      "reactants": ["LiEC_minus_RO1"],
                      "products": ["C2H4", "LiCO3_minus"]},
                     {"ts": "LiEC_RO1__Li2CO3___LEDC_minus_Li_plus",
                      "reactants": ["LEDC_minus_Li_plus"],
                      "products": ["LiEC_RO1", "Li2CO3"]},
                     {"ts": "LiEC_RO1__OH_minus___LEMC_minus",
                      "reactants": ["LEMC_minus"],
                      "products": ["LiEC_RO1", "OH_minus"]},
                     {"ts": "LEMC__Li___DLEMC__H",
                      "reactants": ["LEMC_minus_Li_plus"],
                      "products": ["DLEMC", "H"]},
                     {"ts": "LEMC_minus_Li_2minus___CO2_minus__OC2H4OH_minus",
                      "reactants": ["LEMC_minus_Li_2minus"],
                      "products": ["CO2_minus", "EG_minus_H_minus"]},
                     {"ts": "LEDC__OH_minus___tetrahedral_complex",
                      "reactants": ["tetrahedral_complex"],
                      "products": ["LEDC", "OH_minus"]},
                     {"ts": "LEDC_minus___LiCO3_minus__LiEC_RO1",
                      "reactants": ["LEDC_minus"],
                      "products": ["LiCO3_minus", "LiEC_RO1"]},
                     {"ts": "LEDC_minus___LiCO3_minus__LiEC_RO1",
                      "reactants": ["LEDC_minus"],
                      "products": ["LiCO3", "LiEC_minus_RO1"]},
                     {"ts": "LiEC_shld___CO__Li(OCH2)2_minus",
                      "reactants": ["LiEC_RO_shoulder"],
                      "products": ["CO", "Li_(OCH2)2_minus"]},
                     {"ts": "LiCO3__LiEC___LEDC",
                      "reactants": ["LEDC"],
                      "products": ["LiCO3_minus", "LiEC_plus"]},
                     {"ts": "LiCO3__EC___LEDC_minus_Li",
                      "reactants": ["LEDC_minus_Li"],
                      "products": ["LiCO3_minus", "EC"]},
                     {"ts": "LiEC_plus_bi__OH_minus___LEMC",
                      "reactants": ["LEMC"],
                      "products": ["OH_minus", "LiEC_plus"]},
                     {"ts": "LEMC_minus___LiCO2__EG_minus_H_minus",
                      "reactants": ["LEMC_minus"],
                      "products": ["CO2_minus", "LiEG_minus_H"]},
                     {"ts": "LEMC_minus___LiCO2__EG_minus_H_minus",
                      "reactants": ["LEMC_minus"],
                      "products": ["LiCO2", "EG_minus_H_minus"]},
                     {"ts": "DLEMC_minus___LiCO2__LiOC2H4O_minus",
                      "reactants": ["DLEMC_minus"],
                      "products": ["CO2_minus", "(LiOCH2)2"]},
                     {"ts": "DLEMC_minus___LiCO2__LiOC2H4O_minus",
                      "reactants": ["DLEMC_minus"],
                      "products": ["LiCO2", "Li_(OCH2)2_minus"]},
                     {"ts": "LEDC_minus___CO2_minus__DLEMC",
                      "reactants": ["LEDC_minus"],
                      "products": ["LiCO2", "DLEMC_minus_Li"]},
                      {"ts": "LEDC_minus___CO2_minus__DLEMC",
                      "reactants": ["LEDC_minus"],
                      "products": ["CO2_minus", "DLEMC"]},
                     {"ts": "tetrahedral_complex___HCO3_minus__DLEMC",
                      "reactants": ["tetrahedral_complex"],
                      "products": ["HCO3_minus", "DLEMC"]},
                     {"ts": "tetrahedral_complex___HCO3_minus__DLEMC",
                      "reactants": ["tetrahedral_complex"],
                      "products": ["LiHCO3", "DLEMC_minus_Li"]},
                     {"ts": "HCO3_minus__DLEMC___LiCO3_minus__LEMC",
                      "reactants": ["HCO3_minus", "DLEMC"],
                      "products": ["LiCO3_minus", "LEMC"]},
                     {"ts": "HCO3_minus__DLEMC___LiCO3_minus__LEMC",
                      "reactants": ["LiHCO3", "DLEMC_minus_Li"],
                      "products": ["LiCO3_minus", "LEMC"]},
                     {"ts": "LEDC__OH_minus___LiCO3_minus__LEMC",
                      "reactants": ["OH_minus", "LEDC"],
                      "products": ["LiCO3_minus", "LEMC"]},
                     {"ts": "HCO3_minus__H___H2__CO3_minus",
                      "reactants": ["HCO3_minus", "H"],
                      "products": ["H2", "CO3_minus"]},
                     {"ts": "LiEC_minus___LiEC_RO_shoulder",
                      "reactants": ["LiEC_minus"],
                      "products": ["LiEC_RO_shoulder"]},
                     {"ts": "LiEC__OH_minus___LEMC_minus",
                      "reactants": ["LiEC", "OH_minus"],
                      "products": ["LEMC_minus"]},
                     {"ts": "LEMC_minus__H___DLEMC_minus_Li__H2",
                      "reactants": ["LEMC_minus", "H"],
                      "products": ["DLEMC_minus_Li", "H2"]},
                     {"ts": "HCO3_minus__EC___HLEDC_minus_Li",
                      "reactants": ["HCO3_minus", "EC"],
                      "products": ["HLEDC_minus_Li"]},
                     {"ts": "HCO3_minus__LiEC_plus___HLEDC",
                      "reactants": ["HCO3_minus", "LiEC_plus"],
                      "products": ["HLEDC"]},
                     {"ts": "HLEDC_minus___LiEC_RO__HCO3_minus",
                      "reactants": ["HLEDC_minus"],
                      "products": ["LiEC_RO1", "HCO3_minus"]},
                     {"ts": "HLEDC__OH_minus___HCO3_minus__LEMC",
                      "reactants": ["HLEDC", "OH_minus"],
                      "products": ["HCO3_minus", "LEMC"]},
                     {"ts": "LiEC_RO__LiHCO3___HLEDC_minus_Li_plus",
                      "reactants": ["LiEC_RO1", "LiHCO3"],
                      "products": ["HLEDC_minus_Li_plus"]},
                     {"ts": "tetrahedral_complex_h___HCO3_minus__LEMC",
                      "reactants": ["tetrahedral_complex_h"],
                      "products": ["HCO3_minus", "LEMC"]},
                     ]

        # Barrierless
        rxns_barrierless = [{"reactants": ["LiOH", "CO2"],
                             "products": ["LiHCO3"]},
                            {"reactants": ["CO2_minus", "LiCO2"],
                             "products": ["LiCO3_minus", "CO"]},
                            {"reactants": ["LiCO2", "LiCO2"],
                             "products": ["Li2CO3", "CO"]},
                            {"reactants": ["CO2_minus", "CO2_minus"],
                             "products": ["CO3_2minus", "CO"]},
                            {"reactants": ["CO2_minus", "LiCO2"],
                             "products": ["Li_oxalate_minus_Li"]},
                            {"reactants": ["LiCO2", "LiCO2"],
                             "products": ["Li_oxalate"]},
                            {"reactants": ["CO2_minus", "CO2_minus"],
                             "products": ["oxalate"]},
                            {"reactants": ["tetrahedral_complex_h"],
                             "products": ["HLEDC", "OH_minus"]},
                            {"reactants": ["LEMC", "OH_minus"],
                             "products": ["H2O", "DLEMC_minus_Li"]},
                            {"reactants": ["OH_minus", "CO2"],
                             "products": ["HCO3_minus"]},
                            {"reactants": ["H2O_minus"],
                             "products": ["OH_minus", "H"]},
                            {"reactants": ["LiH2O"],
                             "products": ["LiOH", "H"]},
                            {"reactants": ["(LiOCH2)2", "CO2"],
                             "products": ["DLEMC_minus"]},
                            {"reactants": ["Li_(OCH2)2_minus", "CO2"],
                             "products": ["DLEMC_minus_Li"]},
                            {"reactants": ["DLEMC", "CO2"],
                             "products": ["LEDC"]},
                            {"reactants": ["DLEMC_minus_Li", "CO2"],
                             "products": ["LEDC_minus_Li"]},
                            {"reactants": ["LiEC_RO1", "LiEC_RO1"],
                             "products": ["LEDC", "C2H4"]},
                            {"reactants": ["H", "H"],
                             "products": ["H2"]}]

        print("ADDING REACTIONS WITH TS")
        for reaction in ts_reacts:
            if reaction["ts"] in ts_normal:
                thermo = get_thermo(reaction, mols, mapping, ts_normal)
            elif reaction["ts"] in ts_lowlot:
                data = ts_lowlot[reaction["ts"]]
                thermo = {"dg": data["dg"],
                          "forward_barrier": data["forward_barrier"],
                          "reverse_barrier": data["reverse_barrier"]}
            else:
                print("PROBLEM - TS not found!", reaction)
                continue

            if reaction["ts"] == "LiEC_bi___LiEC_RO1":
                # TODO: WHAT IS THE CORRECT BARRIER TO USE?
                # Tried 0.5, 0.3 previously
                thermo["forward_barrier"] = 0.4

            reactants = sorted([mapping[r] for r in reaction["reactants"]])
            if len(reactants) == 1:
                reactants.append(-1)
                num_reactants = 1
            else:
                num_reactants = 2
            products = sorted([mapping[p] for p in reaction["products"]])
            if len(products) == 1:
                products.append(-1)
                num_products = 1
            else:
                num_products = 2

            if thermo["forward_barrier"] < 0 or thermo["reverse_barrier"] < 0:
                rxns_barrierless.append(reaction)
                print("BARRIERLESS", (reactants[0], reactants[1], products[0], products[1]))
                continue

            k_forward = eyring(thermo["forward_barrier"])
            k_reverse = eyring(thermo["reverse_barrier"])

            if (reactants[0], reactants[1], products[0], products[1]) not in reaction and (reactants[0], reactants[1], products[0], products[1]) not in forbidden_reactions:
                reactions.add((reactants[0], reactants[1], products[0], products[1]))

                # Forward reaction
                rn_cur.execute(insert_reaction,
                               (reaction_index,
                                num_reactants,
                                num_products,
                                reactants[0],
                                reactants[1],
                                products[0],
                                products[1],
                                k_forward,
                                thermo["dg"],
                                thermo["forward_barrier"],
                                False)
                               )
                reaction_index += 1
                rn_con.commit()
            else:
                print("FORBIDDEN", (reactants[0], reactants[1], products[0], products[1]))

            if (products[0], products[1], reactants[0], reactants[1]) not in reactions and (products[0], products[1], reactants[0], reactants[1]) not in forbidden_reactions:
                reactions.add((products[0], products[1], reactants[0], reactants[1]))

                # Reverse reaction
                rn_cur.execute(insert_reaction,
                               (reaction_index,
                                num_products,
                                num_reactants,
                                products[0],
                                products[1],
                                reactants[0],
                                reactants[1],
                                k_reverse,
                                -1 * thermo["dg"],
                                thermo["reverse_barrier"],
                                False)
                               )
                reaction_index += 1
                rn_con.commit()
            else:
                print("FORBIDDEN", (products[0], products[1], reactants[0], reactants[1]))

            print("REACTION:", reaction["ts"], reaction["reactants"], reaction["products"], thermo["dg"], thermo["forward_barrier"], thermo["reverse_barrier"])
        print("DONE ADDING REACTIONS WITH TS; {} TOTAL REACTIONS".format(reaction_index))

        print("ADDING KNOWN BARRIERLESS REACTIONS")
        for reaction in rxns_barrierless:
            g_rct = sum([mols[mapping[r]]["free_energy"] for r in reaction["reactants"]])
            g_pro = sum([mols[mapping[p]]["free_energy"] for p in reaction["products"]])

            dg = g_pro - g_rct

            # By definition barrierless as written

            reactants = [mapping[r] for r in reaction["reactants"]]
            if len(reactants) == 1:
                reactants.append(-1)
                num_reactants = 1
            else:
                num_reactants = 2
            products = [mapping[p] for p in reaction["products"]]
            if len(products) == 1:
                products.append(-1)
                num_products = 1
            else:
                num_products = 2

            # Reaction always barrierless in provided direction
            barrier_forward = 0.0

            if dg < 0:
                barrier_reverse = abs(dg)
            else:
                barrier_reverse = 0.0

            k_forward = eyring(barrier_forward)
            k_reverse = eyring(barrier_reverse)

            if (reactants[0], reactants[1], products[0], products[1]) not in reaction and (reactants[0], reactants[1], products[0], products[1]) not in forbidden_reactions:
                reactions.add((reactants[0], reactants[1], products[0], products[1]))

                # Forward reaction
                rn_cur.execute(insert_reaction,
                               (reaction_index,
                                num_reactants,
                                num_products,
                                reactants[0],
                                reactants[1],
                                products[0],
                                products[1],
                                k_forward,
                                dg,
                                barrier_forward,
                                False)
                               )
                reaction_index += 1
                rn_con.commit()

            if (products[0], products[1], reactants[0], reactants[1]) not in reactions and (products[0], products[1], reactants[0], reactants[1]) not in forbidden_reactions:
                reactions.add((products[0], products[1], reactants[0], reactants[1]))

                # Reverse reaction
                rn_cur.execute(insert_reaction,
                               (reaction_index,
                                num_products,
                                num_reactants,
                                products[0],
                                products[1],
                                reactants[0],
                                reactants[1],
                                k_reverse,
                                -1 * dg,
                                barrier_reverse,
                                False)
                               )
                reaction_index += 1
                rn_con.commit()

            print("BARRIERLESS:", reaction["reactants"], reaction["products"], dg, k_forward, k_reverse)

        print("DONE ADDING BARRIERLESS REACTIONS; {} TOTAL REACTIONS".format(reaction_index))

        rn_cur.execute(
            insert_metadata,
            (len(mols),
             reaction_index)
        )

        rn_con.commit()
        rn_con.close()

        # Initial state time
        rn_con = sqlite3.connect(os.path.join(base_dir, name, "initial_state.sqlite"))
        rn_cur = rn_con.cursor()
        rn_cur.execute(create_initial_state_table)
        rn_cur.execute(create_trajectories_table)
        rn_cur.execute(create_factors_table)
        rn_cur.execute(create_interrupt_state_table)
        rn_con.execute(create_interrupt_cutoff_table)
        rn_con.commit()

        rn_cur.execute(
            "INSERT INTO factors VALUES (?,?,?)",
            (factor_zero, factor_two, factor_duplicate)
        )

        # initial_state = {ec_ind: 9000,
        #                  li_plus_ind: 600,
        #                  h_ind: 10,
        #                  oh_minus_ind: 10,
        #                  co2_ind: 50}

        initial_state = {liec_ind: 500,
                 ledc_ind: 500,
                 lemc_ind: 500,
                 carbonate_ind: 500,
                 oxalate_ind: 500}

        for i in range(len(mols)):
            rn_cur.execute(
                "INSERT INTO initial_state VALUES (?,?)",
                (i, initial_state.get(i,0)))
            
        # add checkpointing tables
        create_interrupt_state = """
        CREATE TABLE interrupt_state (
                seed                    INTEGER NOT NULL,
                species_id              INTEGER NOT NULL,
                count                   INTEGER NOT NULL
                
            );
        """

        create_interrupt_cutoff = """
        CREATE TABLE interrupt_cutoff (
                    seed                    INTEGER NOT NULL,
                    step                    INTEGER NOT NULL,
                    time                    INTEGER NOT NULL
                    
                );
            """
        
        rn_con = sqlite3.connect(os.path.join(base_dir, name, "initial_state.sqlite"))
        rn_cur = rn_con.cursor()
        rn_cur.execute(create_interrupt_state)
        rn_cur.execute(create_interrupt_cutoff)

        rn_con.commit()
        rn_con.close()